## Lorenz-63 and Chaos
  
- Lorenz-63 model is a system of ODEs that has chaotic solutions
  - see: https://docs.dart.ucar.edu/en/latest/guide/lorenz-63-model.html  or  https://en.wikipedia.org/wiki/Lorenz_system
- simplified mathematical representation of atmospheric convection
- applications in various fields, including laser physics, dynamos, and even electric circuits
- tiny changes in initial conditions yield completely different and unpredictable trajectories
- Edward Lorenz showed that the system switches between the two lobes chaotically

In [ ]:
import numpy as np
from IPython.display import HTML, Image, display
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as animation


### Model (Task 2)

Implement the numerical model for the Lorenz-63 system which is defined as a solution sequence or trajectory $\{\mathbf{z}_n\}_{n\geq0}$ via the iteration:
  $$
  \mathbf{z}_{n+1}=\Psi(\mathbf{z}_n)=\mathbf{z}_n+\Delta t\cdot f(\mathbf{z}_n),\quad \text{with state variable } \mathbf{z}_n=({x}_n,{y}_n,{z}_n)^{\rm T},
  $$
  and with:

  - step-size $\Delta t = 0.01$,
  - start position $\mathbf{z}_0=({ x}_0,{ y}_0,{ z}_0)^{\rm T}=\begin{pmatrix}
-0.587 \\
-0.563 \\
16.870
\end{pmatrix}$,
  - vector field
    $f(\mathbf{z})=
    \begin{pmatrix}
\sigma({y}-{x}) \\
{x}(\rho-{z})-{y} \\
{x}{y}-\beta{z}
\end{pmatrix}=
    \begin{pmatrix}
10({y}-{x}) \\
{x}(28-{z})-{y} \\
{x}{y}-\frac83{z}
\end{pmatrix}$.


#### Computation of vector field $f(z)$

In [ ]:
def lorenz(xyz, rho, sigma, beta):
    """
    Parameters
    xyz : array-like, shape (3,)
       Point of interest in three-dimensional space.
    sigma, rho, beta : float
       Parameters defining the Lorenz attractor.

    Returns:
    xyz_dot : array, shape (3,)
       Values of the Lorenz attractor's partial derivatives at *xyz*.
    """
    x, y, z = xyz # unpack
    x_dot = sigma * (y - x) 
    y_dot = x * (rho - z) - y 
    z_dot = x * y - beta * z 
    return np.array([x_dot, y_dot, z_dot])  # needs to be an np.array to allow for math operations on it

#### Computation of Trajectory $\{\mathbf{z}_n\}$

In [ ]:
def calculate_z(z_0, rho, sigma, beta, save_every_nstep, dt, num_steps):
    data = []
    zt = z_0
    
    # store only the observed states
    for i in range(num_steps):
        if i % save_every_nstep == 0:
            data.append(zt)
        zt = zt + dt * lorenz(zt, rho, sigma, beta) # update dynamical system

    return np.array(data)  # convert list to np.array

In [ ]:
# time stepping
dt = 0.01
t_end = 10
num_steps = int(t_end/dt)
save_every_nstep = 2
t = np.linspace(0, t_end, num_steps//save_every_nstep)  # '//' integer division

# Set the initial conditions
z_0 = np.array([-0.587, -0.563, 16.870])

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0/3
curve = calculate_z(z_0, rho, sigma, beta,
                    save_every_nstep, dt, num_steps)
## if needed, store the resulting reference trajectory in the csv file for later use
# np.savetxt('lorenz_data.csv', curve, delimiter=',')

In [ ]:
x = curve[:, 0]
y = curve[:, 1]
z = curve[:, 2]
curve.shape

In [ ]:
means = np.mean(curve, axis=0)
stds = np.std(curve, axis=0)

for label, mean, std in zip(['x', 'y', 'z'], means, stds):
    print(f"Mean {label}: {mean}, Std {label}: {std}")

#### Plotting the Lorenz Attractor

**x-component**

In [ ]:
fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot()
ax.plot(t, x)
ax.set_xlabel("t")
ax.set_ylabel("X")
ax.grid(True)
ax.set_title(f"Lorenz Attractor | rho={rho}  sigma={sigma}  beta={beta:.3f}")
plt.show()

**3D-Plot**

In [ ]:
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(projection='3d')
# modify ax.get_proj method: scale the x and y by 0.9 and 0.7 of the projection matrix
ax.get_proj = lambda: Axes3D.get_proj(ax) @ np.diag([0.9, 0.7, 1, 1])  # 
ax.plot(x, y, z, lw=0.9, c='k')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title(f"Lorenz Attractor | rho={rho}  sigma={sigma}  beta={beta:.3f}")
plt.show()

**as animation**

In [ ]:
change = 0.01
z_0_changed = z_0 + change
curve2 = calculate_z(z_0_changed, rho, sigma, beta,
                     save_every_nstep, dt, num_steps)
x2 = curve2[:, 0]
y2 = curve2[:, 1]
z2 = curve2[:, 2]

# Create figure
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': '3d'}, figsize=(12, 6))

# Initialize plots
line1, = ax.plot([], [], [], lw=1, c='red', label=f"z0+{change}")
point1, = ax.plot([], [], [], 'ro', markersize=5)
point2, = ax.plot([], [], [], 'ko', markersize=5)

ax.plot(x, y, z, lw=0.5, c='gray', ls='dashed', label=f"z0")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_title(f"rho={rho}  sigma={sigma}  beta={beta:.3f}")
plt.legend()

# Initialize animation function
def init():
    line1.set_data([], [])
    line1.set_3d_properties([])
    point1.set_data([], [])
    point1.set_3d_properties([])
    point2.set_data([], [])
    point2.set_3d_properties([])
    return line1, point1

# Update function
def update(i):
    line1.set_data(x2[:i], y2[:i])
    line1.set_3d_properties(z2[:i])
    point1.set_data([x2[i]], [y2[i]])
    point1.set_3d_properties([z2[i]])
    point2.set_data([x[i]], [y[i]])
    point2.set_3d_properties([z[i]])
    return line1, point1

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(z), init_func=init,
                              interval=20, blit=True)

plt.close()  # finish/close animation

## if ffmpeg installed
## Display the animation
# HTML(ani.to_html5_video())

ani.save(filename="lorenz63.gif", writer="pillow")
display(Image(filename="lorenz63.gif"))

 $$\text{RMSE} = \sqrt{\frac{1}{N}\sum_{n=1}^N\biggl(z^\ast_n-z_n\biggr)^2}$$

In [ ]:
rmse = np.sqrt( np.mean( (curve2 - curve)**2 ) )
print(f"RMSE = {rmse}")